In [1]:
#in the absence of known webapi which can be used to retrieve the data , I am using web scrapping
#to get the corporate bond info. getting it from a webapi which will return the bond yields will be much 
#easier

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', 200)

# URL of the source
url = 'https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/gbp-bonds'

# Load data from URL
tables = pd.read_html(url)
# Extract the table of interest (in this case, the first table on the page)
df = tables[0]
df=df.drop(columns=['Actions'])
df.head(10)



,Issuer,Coupon (%),Maturity,Price
0,3i Group GBP | XS0104440986 | 0924597,5.750,3 December 2032,99.575
1,A2D Funding plc GBP | XS1103286305 | BQ8NZW9,4.500,30 September 2026,94.925
2,Abrdn Asia Focus Plc GBX | GB00BFZ0WT29 | BFZ0WT2,2.250,31 May 2025,95.250
3,Allied Irish Banks plc GBP | XS0435957682 | BFMTY18,0.000,25 June 2035,61.000
4,Alpha Plus Holdings plc GBP | XS1379593566 | BZ5ZT30,5.000,31 March 2024,97.925
5,Anglian Water Services Financing plc GBP | XS0093312550 | 0482976,6.625,15 January 2029,103.900
6,Aviva plc GBP | XS1488459485 | BDF5PP0,4.375,12 September 2049,85.350
7,Aviva plc GBP | XS0138717441 | 3106518,6.125,14 November 2036,100.450
8,B.A.T. International Finance GBP | XS0468426266 | B5KP6X4,6.000,24 November 2034,90.875
9,B.A.T. International Finance plc GBP | XS0522408599 | B3RS4H2,5.750,5 July 2040,82.250


In [3]:
df_issuer = df
print(df_issuer['Issuer'])
# Split "issuer" column by "|" character
df_issuer[['Name', 'ISIN', 'SEDOL']] = df_issuer['Issuer'].str.split('|', expand=True)

# Remove leading/trailing whitespace from the newly created columns
df_issuer['Name'] = df_issuer['Name'].str.strip()
df_issuer['ISIN'] = df_issuer['ISIN'].str.strip()
df_issuer['SEDOL'] = df_issuer['SEDOL'].str.strip()

# Drop the original "issuer" column
df_issuer.drop('Issuer', axis=1, inplace=True)

# Get a list of all column names except "Name", "ISIN", and "SEDOL"
other_columns = [col for col in df_issuer.columns if col not in ['Name', 'ISIN', 'SEDOL']]

# Reorder columns with "Name", "ISIN", and "SEDOL" as the first columns, followed by the remaining columns
df_issuer = df_issuer[['Name']+other_columns+['ISIN','SEDOL']]

# Display the updated DataFrame with additional columns
print(df_issuer)


0                            3i Group GBP | XS0104440986 | 0924597
1                     A2D Funding plc GBP | XS1103286305 | BQ8NZW9
2                Abrdn Asia Focus Plc GBX | GB00BFZ0WT29 | BFZ0WT2
3              Allied Irish Banks plc GBP | XS0435957682 | BFMTY18
4             Alpha Plus Holdings plc GBP | XS1379593566 | BZ5ZT30
                                  ...                             
124         Vanquis Banking Group plc GBP | XS1209091856 | BWNH4K9
125                    Vodafone Group GBP | XS0158715713 | 3230097
126                    Vodafone Group GBP | XS0181816652 | 3388361
127    West Bromwich Building Society GBP | GB00B0CX2M20 | B0CX2M2
128        Yorkshire Building Society GBP | XS0498549194 | B5674L8
Name: Issuer, Length: 129, dtype: object
                                   Name  Coupon (%)           Maturity  \
0                          3i Group GBP       5.750    3 December 2032   
1                   A2D Funding plc GBP       4.500  30 September 2026   


In [4]:
bond_underlyings=df_issuer['SEDOL'].values
bond_underlyings

array(['0924597', 'BQ8NZW9', 'BFZ0WT2', 'BFMTY18', 'BZ5ZT30', '0482976',
       'BDF5PP0', '3106518', 'B5KP6X4', 'B3RS4H2', 'BYX4LY7', '0087177',
       '0087025', '3079784', '3149397', 'BYM5X34', 'B0SRM51', 'B19ZPK7',
       'B19ZMT5', '0379722', 'B1YYD83', 'BL6THL7', '0678087', 'BFWXVY2',
       'BDR7W97', 'B04X7D6', 'BYVZGF8', 'BFXW063', 'BFXWHQ2', 'B3XR428',
       'B1YWFR0', '0301208', '3432451', '3304323', 'B3F8XJ9', 'BMFTNK6',
       'B7M3NB7', 'BR4R5Y4', 'BTL1KW6', '0347202', 'B2NMK95', 'B0KCMD2',
       '3115912', 'B1VTTZ0', '0484110', '0039491', '0524287', '3414716',
       '0262468', '3227077', 'B3CPYK1', 'B4Y0FV9', 'BGDQ0W8', 'B03HC45',
       '3342561', '3235382', 'B57N6P3', 'B3D0FP7', 'BJYHVQ2', 'BP95743',
       '0510491', '0365237', 'BD20Y17', 'BL9ZVL0', 'B434V14', '0190536',
       'B3MGZV2', 'B3Q4VK3', '3167913', 'BYYTRZ4', 'B92X2L4', 'BG88V04',
       '0491938', '3068157', '0134653', '0877378', '0631530', 'B908RK5',
       '0637152', '0636137', '0877505', 'B0712W1', 

In [5]:
%%capture
%run gilts_yield_calculator.ipynb

In [6]:
def get_coupon_frequancy(frequency):
    switch_dict = {
        "Annually": 1,
        "Bi-annually": 2,
        "Quarterly": 4
    }
    return switch_dict.get(frequency, None)


In [7]:
# Define a function to generate link HTML
def generate_link(url):
    return f'<a href="{url}" target="_blank">{url}</a>'



In [8]:
b_df = pd.DataFrame(columns = ['SEDOL', 'CouponFreq'])



In [9]:
def add_bond_to_data_frame(bond_underlying: str, concate_to_df):
    print('trying to scrap bond data {} from url '.format(bond_underlying))
    url_hl_search = f'https://www.hl.co.uk/shares/shares-search-results/{bond_underlying}'
    df = pd.read_html(url_hl_search)[0]
    df = df.T
    df.reset_index(drop=True, inplace=True)
    df.columns = df.iloc[0].str.rstrip(':')
    df.drop(0, inplace=True)
    df['Coupon_Freq'] = df['Coupon frequency'].map(get_coupon_frequancy)
    df['SEDOL'] = bond_underlying
    df = df[['Coupon_Freq', 'SEDOL']]
    concate_to_df = pd.concat([concate_to_df, df], ignore_index=False)
    return concate_to_df

In [10]:
bond_underlying = 'B03HC45'
#a = [add_bond_to_data_frame(b,b_df) for b in bond_underlyings]
#b_df=add_bond_to_data_frame(bond_underlying,b_df)
#b_df=b_df.style.format({'URL': generate_link})
#b_df



In [11]:
get_bond_page= (lambda bond_underlying : 
                   f'https://www.hl.co.uk/shares/shares-search-results/{bond_underlying}')

In [12]:
import asyncio
import aiohttp
import pandas as pd
import nest_asyncio
   
nest_asyncio.apply()  # Run this line if using Jupyter Notebook
async def fetch_data(bond_underlying: str):
    async with aiohttp.ClientSession() as session:
        url_hl_search = get_bond_page(bond_underlying)           
        async with session.get(url_hl_search) as response:
            html = await response.text()
            df= pd.read_html(html,flavor='bs4')[0]
            df = df.transpose()
            df.reset_index(drop=True, inplace=True)
            df.columns = df.iloc[0].str.rstrip(':')
            df.drop(0, inplace=True)
            df['CouponFreq'] = df['Coupon frequency'].map(get_coupon_frequancy)
            df['SEDOL'] = bond_underlying
            df = df[['CouponFreq', 'SEDOL']]
            return df

async def get_bond_coupon_freq(bond_underlying: str):
    try:
        print('trying to scrape bond data {} from url '.format(bond_underlying))
        df = await fetch_data(bond_underlying)
        return df
    except Exception as e:        
        print("error when calling bond {} m exception \t {}".format(bond_underlying,e))

async def main():
    global b_df
    tasks = [get_bond_coupon_freq(bond_underlying) for bond_underlying in bond_underlyings]
    results = await asyncio.gather(*tasks)
    b_df= pd.concat([b_df]+results)

await main()


trying to scrape bond data 0924597 from url 
trying to scrape bond data BQ8NZW9 from url 
trying to scrape bond data BFZ0WT2 from url 
trying to scrape bond data BFMTY18 from url 
trying to scrape bond data BZ5ZT30 from url 
trying to scrape bond data 0482976 from url 
trying to scrape bond data BDF5PP0 from url 
trying to scrape bond data 3106518 from url 
trying to scrape bond data B5KP6X4 from url 
trying to scrape bond data B3RS4H2 from url 
trying to scrape bond data BYX4LY7 from url 
trying to scrape bond data 0087177 from url 
trying to scrape bond data 0087025 from url 
trying to scrape bond data 3079784 from url 
trying to scrape bond data 3149397 from url 
trying to scrape bond data BYM5X34 from url 
trying to scrape bond data B0SRM51 from url 
trying to scrape bond data B19ZPK7 from url 
trying to scrape bond data B19ZMT5 from url 
trying to scrape bond data 0379722 from url 
trying to scrape bond data B1YYD83 from url 
trying to scrape bond data BL6THL7 from url 
trying to 

In [13]:
b_df

,SEDOL,CouponFreq
1,0924597,2
1,BQ8NZW9,2
1,BFZ0WT2,2
1,BFMTY18,1
1,BZ5ZT30,2
1,0482976,1
1,BDF5PP0,2
1,3106518,1
1,B5KP6X4,1
1,B3RS4H2,1


In [14]:
df_corp_bonds = pd.merge(df_issuer,b_df, on='SEDOL', how='outer')


In [15]:
df_corp_bonds=df_corp_bonds.rename(columns={'Coupon (%)': 'Coupon'})
df_corp_bonds=df_corp_bonds.dropna()
df_corp_bonds

,Name,Coupon,Maturity,Price,ISIN,SEDOL,CouponFreq
0,3i Group GBP,5.750,3 December 2032,99.575,XS0104440986,0924597,2
1,A2D Funding plc GBP,4.500,30 September 2026,94.925,XS1103286305,BQ8NZW9,2
2,Abrdn Asia Focus Plc GBX,2.250,31 May 2025,95.250,GB00BFZ0WT29,BFZ0WT2,2
3,Allied Irish Banks plc GBP,0.000,25 June 2035,61.000,XS0435957682,BFMTY18,1
4,Alpha Plus Holdings plc GBP,5.000,31 March 2024,97.925,XS1379593566,BZ5ZT30,2
5,Anglian Water Services Financing plc GBP,6.625,15 January 2029,103.900,XS0093312550,0482976,1
6,Aviva plc GBP,4.375,12 September 2049,85.350,XS1488459485,BDF5PP0,2
7,Aviva plc GBP,6.125,14 November 2036,100.450,XS0138717441,3106518,1
8,B.A.T. International Finance GBP,6.000,24 November 2034,90.875,XS0468426266,B5KP6X4,1
9,B.A.T. International Finance plc GBP,5.750,5 July 2040,82.250,XS0522408599,B3RS4H2,1


In [16]:
df_corp_bonds["Maturity"] = pd.to_datetime(df_corp_bonds["Maturity"], format="%d %B %Y")
df_corp_bonds['Ttm'] = df_corp_bonds["Maturity"].apply(lambda maturity: calculate_time_to_maturity(maturity))
df_corp_bonds

TypeError: unsupported operand type(s) for -: 'Timestamp' and 'datetime.date'

In [ ]:
COUPON_EX_DATE_OFFSET=2 # how many business dates prior to date offset the Ex date is

df_corp_bonds['AccruedInterest']=df_corp_bonds.apply(lambda x: calculate_accrued_interest(x['Coupon'],FACE_VALUE,x['Ttm'],x['CouponFreq'],x['NextCouponDate'], COUPON_EX_DATE_OFFSET),axis=1)
df_corp_bonds['DirtyPrice']=df_corp_bonds.apply(lambda x: calculate_dirty_price(x['Price'],x['AccruedInterest']),axis=1)
df_corp_bonds['DirtyYield'] = df_corp_bonds.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['DirtyPrice'],x['CouponFreq']), axis=1)
df_corp_bonds['CleanYield'] = df_corp_bonds.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['Price'],x['CouponFreq']), axis=1)

In [ ]:

df_less_than_Ttm = df_corp_bonds[df_corp_bonds['Ttm'] < 5]
df_less_than_Ttm=df_less_than_Ttm.sort_values("DirtyYield", ascending=False)
df_less_than_Ttm['URL']=df_less_than_Ttm['SEDOL'].apply(lambda sedol: get_bond_page(sedol))
cols = ['Name','URL','SEDOL','Coupon', 'Maturity', 'Price', 'Ttm','DirtyPrice', 'DirtyYield', 'CleanYield']
df_less_than_Ttm = df_less_than_Ttm[cols]
df_less_than_Ttm.style.format({'URL': generate_link})
df_less_than_Ttm

In [ ]:
get_bond_page()